In [1]:
import xarray as xr
import numpy as np
import xoak
import pyfesom2 as pf
import matplotlib.pyplot as plt
import cmocean

/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/climatology.py:14: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater as sw


In [2]:
# Load mesh and data
meshpath = '/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel'
mesh = pf.load_mesh(meshpath, abg=[0, 0, 0], usepickle=False)
ds_mesh = xr.open_dataset(f"{meshpath}/fesom.mesh.diag.nc").set_coords(["lat", "lon"])
ds_mesh = ds_mesh.assign_coords(node_id=(("nod2",), range(ds_mesh.sizes["nod2"])))

# Load real data file, replace 'your_data_file.nc' and 'variable_name' with actual paths and names
data_file = xr.open_dataset('/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel/u.fesom.1958.nc')


/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:189: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(
/gxfs_home/geomar/smomw662/.local/lib/python3.11/site-packages/pyfesom2/load_mesh_data.py:200: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_content = pd.read_csv(


Load 2d part of the mesh in 0 second(s)


In [3]:
data_file

<xarray.Dataset> Size: 1GB
Dimensions:  (nz1: 40, time: 365, elem: 21120)
Coordinates:
  * nz1      (nz1) float64 320B 4.519 14.01 24.45 ... 3.459e+03 3.814e+03
  * time     (time) datetime64[ns] 3kB 1958-01-01T23:50:00 ... 1958-12-31T23:...
Dimensions without coordinates: elem
Data variables:
    u        (time, nz1, elem) float32 1GB ...
Attributes: (12/18)
    Conventions:                         UGRID-1.0
    FESOM_model:                         FESOM2
    FESOM_website:                       fesom.de
    FESOM_git_SHA:                       ab6d892b
    FESOM_MeshPath:                      /albedo/work/projects/p_clidyn_work/...
    FESOM_mesh_representative_checksum:  
    ...                                  ...
    FESOM_use_floatice:                  0
    FESOM_whichEVP:                      0
    FESOM_evp_rheol_steps:               120
    FESOM_opt_visc:                      5
    FESOM_use_wsplit:                    0
    FESOM_autorotate_back_to_geo:        0

In [4]:
data_fesom = data_file['u'].isel(time=-1)  # Select the last time step or another specific time


In [5]:
data_fesom

<xarray.DataArray 'u' (nz1: 40, elem: 21120)> Size: 3MB
[844800 values with dtype=float32]
Coordinates:
  * nz1      (nz1) float64 320B 4.519 14.01 24.45 ... 3.459e+03 3.814e+03
    time     datetime64[ns] 8B 1958-12-31T23:50:00
Dimensions without coordinates: elem
Attributes:
    description:  zonal velocity
    long_name:    zonal velocity
    units:        m/s
    location:     face
    mesh:         fesom_mesh

In [6]:
# Set up interpolation grid
lon_bds = (ds_mesh.lon.min().data[()], np.ceil(ds_mesh.lon.max().data[()]))
lat_bds = (ds_mesh.lat.min().data[()], np.ceil(ds_mesh.lat.max().data[()]))
lon_grid = xr.DataArray(np.linspace(*lon_bds, 101), dims=("lon_grid",), name="lon_grid")
lat_grid = xr.DataArray(np.linspace(*lat_bds, 101), dims=("lat_grid",), name="lat_grid")
lon_grid_selector, lat_grid_selector = xr.broadcast(lon_grid, lat_grid)

# # Set up xoak index for spatial selection
# ds_mesh.xoak.set_index(["lat", "lon"], "sklearn_geo_balltree")
# nodes_grid = ds_mesh.xoak.sel(lon=lon_grid_selector, lat=lat_grid_selector).node_id

# # Interpolate real data onto grid
# data_grid = data_fesom.isel(nod2=nodes_grid)


In [ ]:
mesh.x2

In [22]:
data_fesom.dims

('nz1', 'elem')

In [23]:
### 
#/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel
meshdiag = xr.open_mfdataset('/gxfs_work/geomar/smomw662/fesom_parcels_001/data/channel/fesom.mesh.diag.nc')
elem_n = meshdiag.dims["elem"]

xx2 = np.zeros(shape=(elem_n)) 
yy2 = np.zeros(shape=(elem_n))

for i in np.arange(0,elem_n):
    xx2[i]=mesh.x2[mesh.elem[i,:]].mean(axis=0)
    yy2[i]=mesh.y2[mesh.elem[i,:]].mean(axis=0)

# Choose correct X and Y for quantities on nodes or elements
if 'nod2' in data_fesom.dims:
    X = meshdiag.lon
    Y = meshdiag.lat
elif 'elem' in data_fesom.dims:
    X = xx2
    Y = yy2
else:
    print('Error!')

/tmp/ipykernel_3003130/1825451284.py:4: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  elem_n = meshdiag.dims["elem"]


In [38]:
import numpy as np
from scipy.interpolate import griddata

# Ensure data_flat, X, and Y are the same shape (21120,)
data_flat = data_fesom.values.flatten()
X = X.flatten()
Y = Y.flatten()

# Combine X and Y into a single array of shape (21120, 2)
coords = np.column_stack((X, Y))

# Define the target grid
lon_grid_selector, lat_grid_selector = np.meshgrid(
    np.linspace(min(X), max(X), 101),
    np.linspace(min(Y), max(Y), 101)
)

# Flatten the target grid for interpolation
target_coords = np.column_stack((lon_grid_selector.flatten(), lat_grid_selector.flatten()))

# Now perform the interpolation
data_grid_flat = griddata(
    coords,       # Original element coordinates
    data_flat,    # Original data values
    target_coords,  # Target grid coordinates
    method='linear'  # or 'cubic'
)

# Reshape to the desired grid shape (101, 101)
data_grid = data_grid_flat.reshape((101, 101))


ValueError: different number of values and points

In [ ]:
# # Interpolate from unstructured elements to structured grid
# from scipy.interpolate import griddata

# data_flat = data_fesom.values.flatten()  # Flatten the data array to match with (lon_elem, lat_elem)
# data_grid = griddata(
#     (X, Y),  # Original element coordinates
#     data_flat,  # Original data values
#     (lon_grid_selector, lat_grid_selector),  # Target grid
#     method='linear'  # or 'cubic' depending on the level of smoothness needed
# )

In [ ]:

# Plotting
plt.figure(figsize=(10, 8))
plt.contourf(data_grid.lon_grid, data_grid.lat_grid, data_grid, levels=100, cmap=cmocean.cm.thermal)
plt.colorbar(label=f"{data_fesom.name} ({data_fesom.units})")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Interpolated Data on Regular Grid")
plt.show()